Les imports pour le projet :

In [38]:
import collections
import csv
import sys
import time
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import pandas as pd

Les set-up pour Selenium :

In [39]:
driver_path = ChromeDriverManager().install()
prefs = {'profile.default_content_setting_values': {'images': 2, 'javascript': 2}}
options = Options()
options.binary_location = r'C:/Program Files/Google/Chrome/Application/chrome.exe'
service = Service(executable_path=driver_path)
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')
options.add_argument("enable-automation")
options.add_argument("--window-size=1920,1080")
options.add_argument("--disable-extensions")
options.add_argument("--dns-prefetch-disable")
options.add_argument("--disable-gpu")
options.add_experimental_option('prefs', prefs)
driver =webdriver.Chrome(options = options, service=service)

Ici j'ai d'abord utilisé beautiful soup pour scrap les noms des produits, j'ai voulu répéter le processus sur les prix mais je ne trouvais pas comment faire. J'ai donc décidé d'utiliser Selenium pour faire cela.

Dans ce bloc je combine les deux méthodes pour optimiser les performances (BS étant beaucoup plus rapide que Selenium)

Ici je travaille que sur la première page de Shein Homme pour tester mon algo.

In [68]:
df = pd.DataFrame(columns=['Nom'])
temp = []

# URL du site à scraper
url = "https://fr.shein.com/Men-Clothing-c-2026.html?ici=fr_tab06navbar04&scici=navbar_MenHomePage~~tab06navbar04~~4~~real_2026~~~~0&src_module=topcat&src_tab_page_id=page_home1673351679865&src_identifier=fc%3DMen%60sc%3DV%C3%8ATEMENTS%60tc%3D0%60oc%3D0%60ps%3Dtab06navbar04%60jc%3Dreal_2026&srctype=category&userpath=category-V%C3%8ATEMENTS&child_cat_id=1970"

# Récupérer le contenu de la page
page = requests.get(url)

# Analyser le contenu de la page avec BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

# Initialiser une liste vide pour stocker les noms de produits
product_names = []

# Trouver tous les éléments qui contiennent les noms des produits
product_name_elements = soup.find_all('a', class_='S-product-item__link_jump')

# Pour chaque élément contenant le nom d'un produit, extraire le nom du produit
for element in product_name_elements:
    product_name = element['aria-label']
    temp.append(product_name)

driver = webdriver.Chrome()

# Ouvrir la page web
driver.get("https://fr.shein.com/Men-Clothing-c-2026.html?ici=fr_tab06navbar04&scici=navbar_MenHomePage~~tab06navbar04~~4~~real_2026~~~~0&src_module=topcat&src_tab_page_id=page_home1673351679865&src_identifier=fc%3DMen%60sc%3DV%C3%8ATEMENTS%60tc%3D0%60oc%3D0%60ps%3Dtab06navbar04%60jc%3Dreal_2026&srctype=category&userpath=category-V%C3%8ATEMENTS&child_cat_id=1970")

try:
    element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.normal-price-ctn__sale-price")))
finally:
    product_price_elements = driver.find_elements(By.CSS_SELECTOR, "span.normal-price-ctn__sale-price")
    product_prices = [float(elem.get_attribute('aria-label').replace("Prix", "").replace("€", "")) for elem in product_price_elements]

df = pd.DataFrame(list(zip(temp, product_prices)), columns=['Nom','Prix'])
driver.quit()

df

,Nom,Prix
0,Homme T-shirt à lettres à blocs de couleurs,9.49
1,Homme Chemise unicolore à bouton,13.99
2,SHEIN Homme T-shirt graphique de slogan,9.49
3,Homme Polo à blocs de couleur,13.99
4,SHEIN Homme T-shirt slogan et floral,9.49
...,...,...
115,Homme T-shirt à lettres,9.99
116,Polo avec blocs de couleurs et zip,13.99
117,Homme T-shirt à lettres à col montant,9.99
118,SHEIN Homme Chemise à imprimé floral à bouton,15.99


Dans le bloc suivant j'utilise Selenium pour retrouver à la fois le nom du produit et le prix associé

In [69]:
driver = webdriver.Chrome()

# Ouvrir la page web
driver.get("https://fr.shein.com/Men-Clothing-c-2026.html?ici=fr_tab06navbar04&scici=navbar_MenHomePage~~tab06navbar04~~4~~real_2026~~~~0&src_module=topcat&src_tab_page_id=page_home1673351679865&src_identifier=fc%3DMen%60sc%3DV%C3%8ATEMENTS%60tc%3D0%60oc%3D0%60ps%3Dtab06navbar04%60jc%3Dreal_2026&srctype=category&userpath=category-V%C3%8ATEMENTS&child_cat_id=1970")

# Attendre que les produits soient chargés
try:
    element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.S-product-item__link_jump")))
finally:
    # Extraire les noms des produits à partir de la page
    product_name_elements = driver.find_elements(By.CSS_SELECTOR, "a.S-product-item__link_jump")
    product_names = [elem.get_attribute('aria-label') for elem in product_name_elements]

try:
    element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.normal-price-ctn__sale-price")))
finally:
    # Extraire les prix des produits à partir de la page
    product_price_elements = driver.find_elements(By.CSS_SELECTOR, "span.normal-price-ctn__sale-price")
    product_prices = [float(elem.get_attribute('aria-label').replace("Prix", "").replace("€", "")) for elem in product_price_elements]

df = pd.DataFrame(list(zip(product_names, product_prices)), columns=['Nom','Prix'])
driver.quit()

df

,Nom,Prix
0,Homme T-shirt à lettres à blocs de couleurs,9.49
1,Homme Chemise unicolore à bouton,13.99
2,SHEIN Homme T-shirt graphique de slogan,9.49
3,Homme Polo à blocs de couleur,13.99
4,SHEIN Homme T-shirt slogan et floral,9.49
...,...,...
115,Homme Chemise à applique lettre en velours côt...,23.49
116,Homme Polo à applique lettre poche à rabat à c...,14.99
117,Homme T-shirt réfléchissant fleuri à motif lettre,10.99
118,Extended Sizes Homme Chemise col à revers à bo...,17.49


Dans le bloc suivant on va utiliser une boucle pour scrapper les 40 pages du catalogue Homme de Shein:

In [106]:
page = 1
temp1 = []
temp2 = []

while page != 40:
    url = f"https://fr.shein.com/Men-Clothing-c-2026.html?ici=fr_tab06navbar04&scici=navbar_MenHomePage~~tab06navbar04~~4~~real_2026~~~~0&src_module=topcat&src_tab_page_id=page_home1673351679865&src_identifier=fc%3DMen%60sc%3DV%C3%8ATEMENTS%60tc%3D0%60oc%3D0%60ps%3Dtab06navbar04%60jc%3Dreal_2026&srctype=category&userpath=category-V%C3%8ATEMENTS&child_cat_id=1970&page={page}"
    driver.get(url)
    print(url)
    print()
    try:
        element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.S-product-item__link_jump")))
    finally:
        # Extraire les noms des produits à partir de la page
        product_name_elements = driver.find_elements(By.CSS_SELECTOR, "a.S-product-item__link_jump")
        product_names = [elem.get_attribute('aria-label') for elem in product_name_elements]
        temp1.append(product_names)
    try:
        element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.normal-price-ctn__sale-price")))
    finally:
        # Extraire les prix des produits à partir de la page
        product_price_elements = driver.find_elements(By.CSS_SELECTOR, "span.normal-price-ctn__sale-price")
        product_prices = [float(elem.get_attribute('aria-label').replace("Prix", "").replace("€", "")) for elem in product_price_elements]
        temp2.append(product_prices)
    page = page + 1

driver.quit()

flat_list1 = [item for sublist in temp1 for item in sublist]
flat_list2 = [item for sublist in temp2 for item in sublist]
df = pd.DataFrame(
    {'Nom': flat_list1,
     'Prix': flat_list2
    })

df.to_csv(r'files/shein.csv', index=False,encoding='utf-8-sig')

df

MaxRetryError: HTTPConnectionPool(host='localhost', port=51166): Max retries exceeded with url: /session/3fa7def950dad5e23a9bfebaebc89d3c/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001BE8FC25870>: Failed to establish a new connection: [WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée'))

Comme je suis fais IP BAN de Shein avec Selenium je continue le travail avec BS afin de pouvoir trouver un moyen de trouver le type d'un produit à partir de son nom.

In [81]:
pages = 1
temp = []

while pages != 40:
    url = f"https://fr.shein.com/Men-Clothing-c-2026.html?ici=fr_tab06navbar04&scici=navbar_MenHomePage~~tab06navbar04~~4~~real_2026~~~~0&src_module=topcat&src_tab_page_id=page_home1673351679865&src_identifier=fc%3DMen%60sc%3DV%C3%8ATEMENTS%60tc%3D0%60oc%3D0%60ps%3Dtab06navbar04%60jc%3Dreal_2026&srctype=category&userpath=category-V%C3%8ATEMENTS&child_cat_id=1970&page={pages}"
    # Récupérer le contenu de la page
    page = requests.get(url)
    print(url)
    # Récupérer le contenu de la page
    page = requests.get(url)

    # Analyser le contenu de la page avec BeautifulSoup
    soup = BeautifulSoup(page.content, 'html.parser')

    # Initialiser une liste vide pour stocker les noms de produits
    product_names = []

    # Trouver tous les éléments qui contiennent les noms des produits
    product_name_elements = soup.find_all('a', class_='S-product-item__link_jump')

    # Pour chaque élément contenant le nom d'un produit, extraire le nom du produit
    for element in product_name_elements:
        product_name = element['aria-label']
        temp.append(product_name)
    print('Total Products scrapped: ',len(temp))
    print()
    pages = pages + 1

df_all = pd.DataFrame(temp, columns=['Nom'])
df_all

https://fr.shein.com/Men-Clothing-c-2026.html?ici=fr_tab06navbar04&scici=navbar_MenHomePage~~tab06navbar04~~4~~real_2026~~~~0&src_module=topcat&src_tab_page_id=page_home1673351679865&src_identifier=fc%3DMen%60sc%3DV%C3%8ATEMENTS%60tc%3D0%60oc%3D0%60ps%3Dtab06navbar04%60jc%3Dreal_2026&srctype=category&userpath=category-V%C3%8ATEMENTS&child_cat_id=1970&page=1
Total Products scrapped:  120

https://fr.shein.com/Men-Clothing-c-2026.html?ici=fr_tab06navbar04&scici=navbar_MenHomePage~~tab06navbar04~~4~~real_2026~~~~0&src_module=topcat&src_tab_page_id=page_home1673351679865&src_identifier=fc%3DMen%60sc%3DV%C3%8ATEMENTS%60tc%3D0%60oc%3D0%60ps%3Dtab06navbar04%60jc%3Dreal_2026&srctype=category&userpath=category-V%C3%8ATEMENTS&child_cat_id=1970&page=2
Total Products scrapped:  239

https://fr.shein.com/Men-Clothing-c-2026.html?ici=fr_tab06navbar04&scici=navbar_MenHomePage~~tab06navbar04~~4~~real_2026~~~~0&src_module=topcat&src_tab_page_id=page_home1673351679865&src_identifier=fc%3DMen%60sc%3DV%C3

In [84]:
df_all

,Nom
0,Homme T-shirt à lettres à blocs de couleurs
1,Homme Chemise unicolore à bouton
2,SHEIN Homme T-shirt graphique de slogan
3,Homme Polo à blocs de couleur
4,SHEIN Homme T-shirt slogan et floral
...,...
4664,Homme Chemise à imprimé aléatoire à bouton
4665,Homme Polo blocs de couleur
4666,Homme T-shirt à lettres à ourlet à rayures
4667,Homme Polo à rayures


In [128]:
shein = pd.read_csv("files/shein.csv")

In [129]:
shein

,Nom,Prix
0,Homme T-shirt à liséré contrastant à bouton,10.49
1,SHEIN Homme T-shirt graphique de slogan,8.99
2,Homme 4 pièces T-shirt unicolore,33.49
3,Homme Polo à blocs de couleur,13.99
4,Homme T-shirt dessin animé,9.99
...,...,...
4663,Extended Sizes Homme T-shirt à lettres à blocs...,8.50
4664,DAZY Homme T-shirt unicolore,10.49
4665,DAZY Homme Chemise à carreaux,17.49
4666,DAZY Homme T-shirt unicolore col rond,14.49


In [150]:
def get_type(x):
    homme = False
    is_there_homme = False
    is_there_shein = False
    SHEIN = False
    product_type = 'Undifined'
    for elem in x.split():
        if homme:
            homme = False
            if not elem.isdigit():
                product_type = elem
                is_there_homme = True
        if elem == 'Homme':
            homme = True

    if not is_there_homme:
        for elem in x.split():
            if SHEIN:
                SHEIN = False
                if not elem.isdigit():
                    product_type = elem
                    is_there_shein = True
            if elem == 'SHEIN':
                SHEIN = True

    if not is_there_homme and not is_there_shein:
        product_type = 'Undifined'
    return product_type

In [151]:
shein['Type'] = shein.Nom.apply(get_type)
shein

,Nom,Prix,Type
0,Homme T-shirt à liséré contrastant à bouton,10.49,T-shirt
1,SHEIN Homme T-shirt graphique de slogan,8.99,T-shirt
2,Homme 4 pièces T-shirt unicolore,33.49,Undifined
3,Homme Polo à blocs de couleur,13.99,Polo
4,Homme T-shirt dessin animé,9.99,T-shirt
...,...,...,...
4663,Extended Sizes Homme T-shirt à lettres à blocs...,8.50,T-shirt
4664,DAZY Homme T-shirt unicolore,10.49,T-shirt
4665,DAZY Homme Chemise à carreaux,17.49,Chemise
4666,DAZY Homme T-shirt unicolore col rond,14.49,T-shirt


In [152]:
shein.Type.value_counts()

T-shirt        2412
Chemise        1274
Polo            587
Undifined       211
Top              89
UNISEX           35
Kimono           18
Tops             16
Homme            14
Maillot           2
Men               2
Kurta             1
Sweat-shirt       1
BASICS            1
X                 1
Jupe              1
Blouse            1
poupée            1
Débardeur         1
Name: Type, dtype: int64